In [15]:
!conda install -c conda-forge folium=0.5.0 --yes

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  51.37 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  36.27 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  38.34 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  48.88 MB/s


In [50]:
import numpy as np
import pandas as pd

# import folium to put the markers on Toronto map
import folium
import requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

from geopy.geocoders import Nominatim

In [11]:
postal_codes_df = pd.read_csv('postal_codes_df_latlong.csv')

In [17]:
postal_codes_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785730,-79.158750
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765690,-79.175256
3,M1G,Scarborough,Woburn,43.768359,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440


In [10]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [18]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(postal_codes_df['Latitude'], postal_codes_df['Longitude'], postal_codes_df['Borough'], postal_codes_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [22]:
CLIENT_ID = 'W1IBZH5RBF2TG0R2OSC4AQUGDSA0ACTQXGRAAW2CILXZHHBN' # your Foursquare ID
CLIENT_SECRET = 'M101F2S53B2C1HQRC04WHDZUI3Y32YZ5C1YKPXQFDEEACKKD' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: W1IBZH5RBF2TG0R2OSC4AQUGDSA0ACTQXGRAAW2CILXZHHBN
CLIENT_SECRET:M101F2S53B2C1HQRC04WHDZUI3Y32YZ5C1YKPXQFDEEACKKD


In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [25]:
toronto_venues = getNearbyVenues(names=postal_codes_df['Neighborhood'],
                                   latitudes=postal_codes_df['Latitude'],
                                   longitudes=postal_codes_df['Longitude']
                                  )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

In [27]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.811525,-79.195517,Canadian Appliance Source Whitby,43.808353,-79.191331,Home Service
1,"Highland Creek, Rouge Hill, Port Union",43.785730,-79.158750,RIGHT WAY TO GOLF,43.785177,-79.161108,Golf Course
2,"Highland Creek, Rouge Hill, Port Union",43.785730,-79.158750,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Guildwood, Morningside, West Hill",43.765690,-79.175256,Homestead Roofing Repair,43.765140,-79.178663,Construction & Landscaping
4,"Guildwood, Morningside, West Hill",43.765690,-79.175256,Heron Park Community Centre,43.768867,-79.176958,Gym / Fitness Center


In [39]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [41]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.00,0.030000,0.000000,0.0,0.010000,0.00,...,0.00,0.00,0.010000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000
1,Agincourt,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.00,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.00,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.00,...,0.00,0.00,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,0.000000
4,"Alderwood, Long Branch",0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.00,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,"Bathurst Manor, Downsview North, Wilson Heights",0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.00,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Bayview Village,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.00,...,0.25,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,"Bedford Park, Lawrence Manor East",0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.00,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Berczy Park,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.015873,0.00,...,0.00,0.00,0.015873,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,"Birch Cliff, Cliffside West",0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.00,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [42]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [95]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Hotel,Gastropub,Steakhouse,American Restaurant,Burger Joint,Restaurant,Bakery,Deli / Bodega
1,Agincourt,Chinese Restaurant,Shopping Mall,Supermarket,Bakery,Hong Kong Restaurant,Discount Store,Badminton Court,Pool,Sushi Restaurant,Shanghai Restaurant
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Pharmacy,Fast Food Restaurant,Farm,Falafel Restaurant,Exhibit,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Hardware Store,Japanese Restaurant,Pharmacy,Pizza Place,Coffee Shop,Sandwich Place,Park,Liquor Store,Beer Store
4,"Alderwood, Long Branch",Sandwich Place,Dance Studio,Gym,Pub,Women's Store,Donut Shop,Dive Bar,Doctor's Office,Dog Run,Dumpling Restaurant


In [93]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 4, 0, 0, 2, 4, 0, 0, 4], dtype=int32)

In [96]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = postal_codes_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.811525,-79.195517,1.0,Home Service,Women's Store,Electronics Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785730,-79.158750,4.0,Bar,Golf Course,Women's Store,Eastern European Restaurant,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Electronics Store,Discount Store
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765690,-79.175256,4.0,Construction & Landscaping,Gym / Fitness Center,Park,Women's Store,Eastern European Restaurant,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant
3,M1G,Scarborough,Woburn,43.768359,-79.217590,0.0,Coffee Shop,Park,Business Service,Korean Restaurant,Women's Store,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440,3.0,Playground,Women's Store,Diner,Farm,Falafel Restaurant,Exhibit,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


In [97]:
toronto_merged.dropna(inplace=True)

In [104]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    cluster = int(cluster)
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster+1), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Cluster #1 - Coffee Shops and Restaurants

In [99]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]].head(10)

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Scarborough,0.0,Coffee Shop,Park,Business Service,Korean Restaurant,Women's Store,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
5,Scarborough,0.0,Restaurant,Indian Restaurant,Grocery Store,Train Station,Dumpling Restaurant,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop
6,Scarborough,0.0,Discount Store,Chinese Restaurant,Hobby Shop,Convenience Store,Department Store,Coffee Shop,Eastern European Restaurant,Doctor's Office,Dog Run,Donut Shop
7,Scarborough,0.0,Intersection,Bus Line,Bakery,Coffee Shop,Soccer Field,Bus Station,Women's Store,Eastern European Restaurant,Dog Run,Donut Shop
8,Scarborough,0.0,Fast Food Restaurant,Furniture / Home Store,Pizza Place,Coffee Shop,Discount Store,Sandwich Place,Liquor Store,Pharmacy,Gourmet Shop,Department Store
10,Scarborough,0.0,Bakery,Gift Shop,Light Rail Station,Electronics Store,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Women's Store
12,Scarborough,0.0,Chinese Restaurant,Shopping Mall,Supermarket,Bakery,Hong Kong Restaurant,Discount Store,Badminton Court,Pool,Sushi Restaurant,Shanghai Restaurant
13,Scarborough,0.0,Pharmacy,Pizza Place,Golf Course,Chinese Restaurant,Thai Restaurant,Coffee Shop,Bus Stop,Fried Chicken Joint,Shopping Mall,Hobby Shop
15,Scarborough,0.0,Chinese Restaurant,Fast Food Restaurant,Coffee Shop,Grocery Store,Electronics Store,Sandwich Place,Pharmacy,Pizza Place,Other Great Outdoors,Construction & Landscaping
17,North York,0.0,Dog Run,Residential Building (Apartment / Condo),Women's Store,Diner,Farm,Falafel Restaurant,Exhibit,Event Space,Ethiopian Restaurant,Electronics Store


## Cluster #2 - Home Services and Women's Store

In [100]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]].head(10)

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Rouge, Malvern",1.0,Home Service,Women's Store,Electronics Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant
63,Roselawn,1.0,Home Service,Women's Store,Electronics Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant


## Cluster #3 - Men's Store & Woman's Store

In [101]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]].head(10)

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,"Bathurst Manor, Downsview North, Wilson Heights",2.0,Men's Store,Women's Store,Fast Food Restaurant,Farm,Falafel Restaurant,Exhibit,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


## Cluster #4 - Playgrounds, Dog Run and Restaurants

In [102]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]].head(10)

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Cedarbrae,3.0,Playground,Women's Store,Diner,Farm,Falafel Restaurant,Exhibit,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
11,"Maryvale, Wexford",3.0,Auto Garage,Convenience Store,Women's Store,Ethiopian Restaurant,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space
48,"Moore Park, Summerhill East",3.0,Restaurant,Tennis Court,Gym,Playground,Convenience Store,Donut Shop,Diner,Discount Store,Dive Bar,Doctor's Office


## Cluster #5 - Bars, Parks, Stores and Restaurants

In [103]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]].head(10)

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Highland Creek, Rouge Hill, Port Union",4.0,Bar,Golf Course,Women's Store,Eastern European Restaurant,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Electronics Store,Discount Store
2,"Guildwood, Morningside, West Hill",4.0,Construction & Landscaping,Gym / Fitness Center,Park,Women's Store,Eastern European Restaurant,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant
9,"Birch Cliff, Cliffside West",4.0,College Stadium,Park,General Entertainment,Gym,Skating Rink,Gym Pool,Event Space,Ethiopian Restaurant,Electronics Store,Dim Sum Restaurant
14,"Agincourt North, L'Amoreaux East, Milliken, St...",4.0,Pharmacy,Fast Food Restaurant,Farm,Falafel Restaurant,Exhibit,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
19,Bayview Village,4.0,Trail,Park,Construction & Landscaping,Dog Run,Electronics Store,Dive Bar,Doctor's Office,Donut Shop,Dumpling Restaurant,Eastern European Restaurant
23,York Mills West,4.0,Speakeasy,Bank,Park,Korean Restaurant,Electronics Store,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant
25,Parkwoods,4.0,Fast Food Restaurant,Park,Food & Drink Shop,Eastern European Restaurant,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Women's Store
31,Downsview West,4.0,Moving Target,Hotel,Park,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Women's Store
34,Victoria Village,4.0,Nail Salon,Park,Intersection,Grocery Store,Dumpling Restaurant,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop
40,East Toronto,4.0,Farmers Market,Italian Restaurant,Music Venue,Park,Bar,Dumpling Restaurant,Doctor's Office,Dog Run,Donut Shop,Eastern European Restaurant
